In [1]:
import sklearn
import numpy as np
import pandas as pd

import torch
cuda_available = torch.cuda.is_available()

from simpletransformers.ner import NERModel, NERArgs

# from ner_eval import collect_named_entities
# from ner_eval import compute_metrics
# from ner_eval import compute_precision_recall_wrapper

# from sklearn_crfsuite import metric
from nervaluate import Evaluator

In [2]:
model_name='bert-base-cased'

In [3]:
train_df =pd.read_csv("..\..\..\Dataset\\simpletransformers\\train.csv")
eval_df =pd.read_csv("..\..\..\Dataset\\simpletransformers\\val.csv")
test_df =pd.read_csv("..\..\..\Dataset\\simpletransformers\\test.csv")

In [4]:
label_list = [
    "O", 
    "B-PER", "I-PER",
    "B-ORG", "I-ORG",
    "B-LOC", "I-LOC",
    "B-PROD", "I-PROD",
    "B-WA", "I-WA",
    "B-EV", "I-EV",
]

In [5]:
# Create a NERModel
model_args=NERArgs()
# model_args.train_batch_size=64
# model_args.overwrite_output_dir=True
model_args = {
    "output_dir": "outputs/",
    "cache_dir": "cache_dir/",

    "fp16": True,
    "fp16_opt_level": "O1",
    "max_seq_length": 512,
    "train_batch_size": 8,
    "gradient_accumulation_steps": 1,
    "eval_batch_size": 8,
    # "num_train_epochs": 1,
    "num_train_epochs": 5,
    "weight_decay": 0,
    "learning_rate": 4e-5,

    "adam_epsilon": 1e-8,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,

    "logging_steps": 50,
    "save_steps": 2000,

    "overwrite_output_dir": True,
    "reprocess_input_data": False,
    "evaluate_during_training": True,

    "do_lower_case": False, #True if using uncased models
}


In [6]:
# model = NERModel('bert', 
#                 'outputs/checkpoint-594-epoch-1',
#                 args=model_args,
#                 labels=label_list,
#                 use_cuda=cuda_available,
#                 )
# # result, model_outputs, wrong_preds = model.eval_model(eval_df)
# # result

In [7]:
model = NERModel('bert',
                model_name,
                args=model_args,
                labels=label_list,
                use_cuda=cuda_available,
                )

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [8]:
model.train_model(train_df,  eval_data=eval_df)

2022-01-08 22:55:39 simpletransformers.ner.ner_model INFO:  Features loaded from cache at cache_dir/cached_train_bert_512_13_4752
Epoch 1 of 5:   0%|          | 0/5 [00:00<?, ?it/s]d:\ner-idn-tweet\env\lib\site-packages\torch\optim\lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
Epochs 0/5. Running Loss:    0.2920: 100%|██████████| 594/594 [02:18<00:00,  4.29it/s]
2022-01-08 22:58:00 simpletransformers.ner.ner_model INFO:  Features loaded from cache at cache_dir/cached_dev_bert_512_13_1188
Epochs 1/5. Running Loss:    0.1502: 100%|██████████| 594

(2970,
 defaultdict(list,
             {'global_step': [594, 1188, 1782, 2000, 2376, 2970],
              'train_loss': [0.29201263189315796,
               0.15019290149211884,
               0.12145843356847763,
               0.0857706144452095,
               0.12670135498046875,
               0.07603207975625992],
              'eval_loss': [0.2755579965426618,
               0.22082056360156743,
               0.2290478620940707,
               0.2455031360630821,
               0.246402929905777,
               0.2695582954700561],
              'precision': [0.7434367541766109,
               0.7091518926689028,
               0.7479338842975206,
               0.7309970384995064,
               0.7330405075646657,
               0.7232015245354931],
              'recall': [0.5479331574318381,
               0.6508355321020228,
               0.6367634124890061,
               0.6512752858399297,
               0.6605101143359718,
               0.6675461741424802],
         

In [9]:
result, model_outputs, y_pred = model.eval_model(test_df)

2022-01-08 23:08:43 simpletransformers.ner.ner_model INFO:  Features loaded from cache at cache_dir/cached_dev_bert_512_13_1485
Running Evaluation: 100%|██████████| 186/186 [00:12<00:00, 14.99it/s]
2022-01-08 23:08:56 simpletransformers.ner.ner_model INFO: {'eval_loss': 0.11826334269003823, 'precision': 0.8663777537016973, 'recall': 0.83969198459923, 'f1_score': 0.8528261642374688}


In [10]:
result

{'eval_loss': 0.11826334269003823,
 'precision': 0.8663777537016973,
 'recall': 0.83969198459923,
 'f1_score': 0.8528261642374688}

In [11]:
df_x_test=test_df.groupby('sentence_id')['words'].apply(list)
x_test=list(df_x_test)

df_y_test=test_df.groupby('sentence_id')['labels'].apply(list)
y_test=list(df_y_test)

In [12]:
predictions, raw_outputs = model.predict(
    # ["@PolsekAlian PolriBerikan inovasi-inovasi Polri yang memudahkan masyarakat, kegiatan pemolisian masyarakat dan seluruh tindakan Kepolisian dalam menghadapi gangguan Kamtibmas, dan lain-lain - Jenderal Polisi Drs. Listyo Sigit Prabowo, https://t.co/f8GZ6VPqPD. - Kapolri #PolriTVRadioPresisi"],
    ["@ShopeeID Bismillah yok menang ���� SHOPEE @ShopeeID #44ShopeeMamamoo #ShopeexMAMAMOO #AdaMamamoodiShopee"], 
    ["@OPPOIndonesia ✍Butuh memori besar 128GB di OPPO A54 buat simpan file-file tugas kuliah dan testimoni online shop -ku ��Semoga oppo A54 bisa menunjang penghasilan keluargaku �� @OPPOIndonesia @nisa_rkt @Bebheey @kus_ica @Erna19_ @Saftriyuni #OPPOA54 https://t.co/bMYIiRb4QF �� 2.921"]
        )

2022-01-08 23:10:23 simpletransformers.ner.ner_model INFO:  Converting to features started.
Running Prediction: 100%|██████████| 1/1 [00:00<00:00, 17.09it/s]


In [13]:
predictions

[[{'@ShopeeID': 'O'},
  {'Bismillah': 'O'},
  {'yok': 'O'},
  {'menang': 'O'},
  {'����': 'O'},
  {'SHOPEE': 'B-PROD'},
  {'@ShopeeID': 'O'},
  {'#44ShopeeMamamoo': 'O'},
  {'#ShopeexMAMAMOO': 'O'},
  {'#AdaMamamoodiShopee': 'O'}]]

In [ ]:
#Now we create the model again using the best estimators

print(sklearn.metrics.classification_report(
    y_test, y_pred, labels=label_list, digits=3
    ))

ValueError: You appear to be using a legacy multi-label data representation. Sequence of sequences are no longer supported; use a binary array or sparse matrix instead - the MultiLabelBinarizer transformer can convert to this format.

In [14]:
evaluator = Evaluator(y_test, y_pred, tags=['LOC', 'PER', 'ORG', 'PROD', 'WA', 'EV'], loader="list")

In [15]:
results, results_per_tag = evaluator.evaluate()

In [16]:
results['partial']

{'correct': 2460,
 'incorrect': 0,
 'partial': 124,
 'missed': 273,
 'spurious': 185,
 'possible': 2857,
 'actual': 2769,
 'precision': 0.9107981220657277,
 'recall': 0.8827441372068603,
 'f1': 0.896551724137931}

In [17]:
results['exact']

{'correct': 2460,
 'incorrect': 124,
 'partial': 0,
 'missed': 273,
 'spurious': 185,
 'possible': 2857,
 'actual': 2769,
 'precision': 0.8884073672806068,
 'recall': 0.8610430521526077,
 'f1': 0.8745111980092429}

In [18]:
import re

temp_l_l=list()
for i in range(len(y_test)):
    temp_l=list()
    print(x_test[i])
    print(len(x_test[i]))
    print("y_test       ",len(y_test[i]),y_test[i])
    print("y_pred       ",len(y_pred[i]),y_pred[i])
    for j in range(len(y_pred[i])):
        temp_l.append((x_test[i][j],y_test[i][j],y_pred[i][j]))
    temp_l_l.append(temp_l)
    print()

['@GyuuPotter', 'iyhh', ',', 'soalnya', 'mwu', 'kerjain', 'tugas', 'jg��']
8
y_test        8 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
y_pred        8 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

['@ShopeeID', 'Bismillah', 'yok', 'menang', '����', 'SHOPEE', '@ShopeeID', '#44ShopeeMamamoo', '#ShopeexMAMAMOO', '#AdaMamamoodiShopee']
10
y_test        10 ['O', 'O', 'O', 'O', 'O', 'B-PROD', 'O', 'O', 'O', 'O']
y_pred        10 ['O', 'O', 'O', 'O', 'O', 'B-PROD', 'O', 'O', 'O', 'O']

['RT', '@kukluxcats', ':', 'Selevel', 'gubernur', 'Jatim', 'aja', 'udah', 'maen', 'BuzzeRp', ',', 'emang', 'udah', 'parah', 'kayanya', 'kompetensi', 'pejabat', 'kita', 'sampe', 'sampe', 'apa', 'apa', 'musti', 'dipromote', 'BuzzeRp', 'https://t.co/kfpglITYzw']
26
y_test        26 ['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
y_pred        26 ['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

In [ ]:
with open("./results_pred.txt", "w") as f:
    str_results = repr(y_pred)
    f.write(str_results)

with open("./results_test_pred.txt", "w",encoding="utf-8") as f:
    str_results = repr(temp_l_l)
    f.write(str_results)

In [18]:
with open("./results_1_batch.txt", "w") as f:
    str_results = repr(results)
    f.write(str_results)

In [19]:
with open("./results_per_tag_1_batch.txt", "w") as f:
    str_results_per_tag = repr(results_per_tag)
    f.write(str_results_per_tag)

In [20]:
import re
for i in range(len(y_test)):
    if len(y_test[i]) != len(y_pred[i]):
        print(x_test[i])
        print(len(x_test[i]))
        print("y_test       ",len(y_test[i]),y_test[i])
        print("y_pred before",len(y_pred[i]),y_pred[i])
        for j in range(len(x_test[i])):
            # if '�' in x_test[i][j]: 
            qmark=re.match(r'^�+$',x_test[i][j])
            if qmark:
                print(qmark)
                y_pred[i].insert(j,'O')
        print("y_pred after ",len(y_pred[i]), y_pred[i])
        print()